# Transforming Data in JSON files into Structured Tables

In this project, we will convert case.json file into three csv files. The definition of three csv files are presented below:

### 1. CuratedOfferOptions.csv

CurationProvider: in quotes

OfferId: in quotes

DealerId: in quotes

UniqueOptionId: in quotes

OptionId: in quotes

IsMobileDealer: without quotes

IsOpen: without quotes

Eta: in quotes

ChamaScore: without quotes

ProductBrand: in quotes

IsWinner: without quotes

MinimumPrice: without quotes

MaximumPrice: without quotes

DynamicPrice: without quotes

FinalPrice: without quotes

DefeatPrimaryReason: in quotes

DefeatReasons: in quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)

### 2. DynamicPriceOptions.csv

Provider: in quotes

OfferId: in quotes

UniqueOptionId: in quotes

BestPrice: without quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)

### 3. DynamicPriceRange.csv

Provider: in quotes

OfferId: in quotes

MinGlobal: without quotes

MinRecommended: without quotes

MaxRecommended: without quotes

DifferenceMinRecommendMinTheory: without quotes

EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)

Let's load the json file into python environment and explore the contents

### Importing Required Libraries

In [17]:
import pandas as pd
import json

In [18]:
df = pd.read_json(path_or_buf = 'case.json')
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


Here we see the data is divided into three columns in the json file, where 'EnquedTimeUtc' displays the date time, 'EventName' provides the event name, and 'Payload' is where event specific data is stored.

In [19]:
df['EventName'].unique()

array(['DynamicPrice_Result', 'CurateOffer_Result'], dtype=object)

Here we see 'DynamicPrice_Result' and 'CurateOffer_Result' as two values for EventName column. We will use 'DynaicPrice_Result' to create 2nd and 3rd .csv file and 'CurateOffer_Result' for 1st .csv file

In [20]:
df.shape

(37, 3)

We have 37 rows and 3 columns of data, let's check to see if there are any null values in the data

In [21]:
df.isnull().sum()

EnqueuedTimeUtc    0
EventName          0
Payload            0
dtype: int64

There are no null values within the data, so now let's move to create the csv files. The first thing to keep in mind is the timezone for all three csv files is same - Brazilian timezone UTC-3. We will convert 'EnqueuedTimeUtc' into required timezone.

In [22]:
df['EnqueuedTimeUtc'] = pd.to_datetime(arg = df['EnqueuedTimeUtc']).dt.tz_convert('Brazil/East')

In [23]:
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


### Dynamic Price Option

In [24]:
dynamic_price_events = df[df['EventName'] == 'DynamicPrice_Result']
dynamic_price_events.shape

(32, 3)

In [25]:
dynamic_price_events.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [41]:
def dynamic_price_option(row):
    
    #Transform the json payload into dynamic price option dataframe
    enqueued_time = row['EnqueuedTimeUtc'].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    result = []
    if payload_row['provider'] == 'ApplyDynamicPricePerOption':
        #Formatting the provider and offer_id with double quotes
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        #Iterating over unique options in the "algorithmOutput" of the payload
        for unique_option in payload_row['algorithmOutput']:
            #Creating a dictionary for each price option
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option['bestPrice'],
                "EnqueuedTimeSp": enqueued_time
            }
            #Appending the price option dictionary to the result list
            result.append(price_option)
    return result
    

In [46]:
lst = dynamic_price_events.apply(dynamic_price_option, axis=1)
flat = [x for xs in lst for x in xs]
dynamic_price_option_payload = pd.DataFrame(flat)

In [47]:
dynamic_price_option_payload

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSp
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/21
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/21
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/21
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21


In [66]:
dynamic_price_option_payload.to_csv("dynamic_price_option_payload.csv", index=False)

### Dynamic Price Range

In [61]:
def dynamic_price_range(row):
    #Transform the json payload into dynamic price range dataframe
    enqueued_time = row['EnqueuedTimeUtc'].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row['Payload']
    payload_row = json.loads(payload_row)
    result = []
    if payload_row['provider'] == 'ApplyDynamicPriceRange':
        provider = f"\"{payload_row['provider']}\""
        offerid = f"\"{payload_row['offerId']}\""
        algorithm_output = payload_row['algorithmOutput']
        price_range = {
            "Provider": provider,
            "OfferId": offerid,
            "MinGlobal": algorithm_output['min_global'],
            "MinRecommended": algorithm_output['min_recommended'],
            "MaxRecommended": algorithm_output['max_recommended'],
            "DifferenceMinRecommendMinTheory": algorithm_output['differenceMinRecommendMinTheory'],
            "EnqueuedTimeSP": enqueued_time
        }
        result.append(price_range)
    return result
    
    

In [64]:
lst1 = dynamic_price_events.apply(dynamic_price_range, axis=1)
flat1 = [x for xs in lst for x in xs]
dynamic_price_range_payload = pd.DataFrame(flat1)

In [65]:
dynamic_price_range_payload

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSp
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21
5,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b7a7b6d1-4dae-7392-5aaf-f3369c29db1d""",93.00,18/08/21
6,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""577e4bbd-f49d-ac23-56a6-e70072a05229""",93.00,18/08/21
7,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""f9b876ab-2590-952f-d69d-5b352ec251f3""",91.35,18/08/21
8,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
9,"""ApplyDynamicPricePerOption""","""00991873-194e-4a6e-89c9-8f68668b6aaa""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21


In [67]:
dynamic_price_range_payload.to_csv("dynamic_price_range_payload.csv", index=False)

### Curated Offer Options

In [68]:
curated_offer_options = df[df['EventName'] == 'CurateOffer_Result']

In [69]:
curated_offer_options

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [72]:
def curated_offer(row):
    #Transform JSON payload into curated offer dataframe
    
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    for el in payload_row:
        curation_provider = f"\"{el['curationProvider']}\""
        offer_id = f"\"{el['offerId']}\""
        dealer_id = f"\"{el['dealerId']}\""
        for option in el["options"]:
            offer_options = {
                "CurationProvider": curation_provider,
                "OfferId": offer_id,
                "DealerId": dealer_id,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                "OptionId": f"\"{option['optionId']}\"",
                "IsMobileDealer": option["isMobileDealer"],
                "IsOpen": option["isOpen"],
                "Eta": f"\"{option['eta']}\"",
                "ChamaScore": option["chamaScore"],
                "ProductBrand": f"\"{option['productBrand']}\"",
                "IsWinner": option["isWinner"],
                "MinimumPrice": option["minimumPrice"],
                "MaximumPrice": option["maximumPrice"],
                "DynamicPrice": option["dynamicPrice"],
                "FinalPrice": option["finalPrice"]
            }
            if "defeatPrimaryReason" in option:
                offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
                offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""
            else:
                offer_options["DefeatPrimaryReason"] = "\"\""
                offer_options["DefeatReasons"] = "\"\""
            offer_options["EnqueuedTimeSP"] = enqueued_time
            result.append(offer_options)
    return result


flatten = [x for xs in curated_offer_options.apply(curated_offer, axis=1) for x in xs]
curated_offers_payload = pd.DataFrame(flatten)

In [74]:
curated_offers_payload.head()

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/21
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""Closed""","""['Closed', 'HasDriverInOffer']""",25/08/21
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.00,198.00,99.95,99.95,"""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",25/08/21
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.00,184.00,93.90,93.90,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21


In [75]:
curated_offers_payload.to_csv("curated_offers_payload.csv")